In [ ]:
import heapq

# Node structure for the Branch and Bound tree
class Node:
    def __init__(self, level, profit, weight, bound):
        self.level = level      # index of item
        self.profit = profit    # current profit
        self.weight = weight    # current weight
        self.bound = bound      # upper bound on profit

    def __lt__(self, other):
        # For max-heap (we use negative bound in heap)
        return self.bound > other.bound


# Function to calculate upper bound on profit
def bound(u, n, W, items):
    if u.weight >= W:
        return 0

    profit_bound = u.profit
    j = u.level + 1
    total_weight = u.weight

    # Include items greedily until capacity allows
    while j < n and total_weight + items[j][1] <= W:
        total_weight += items[j][1]
        profit_bound += items[j][0]
        j += 1

    # Take fraction of the next item (for bound only)
    if j < n:
        profit_bound += (W - total_weight) * items[j][0] / items[j][1]

    return profit_bound


# Main function to solve 0/1 Knapsack using LC Branch and Bound
def knapsackLC(items, W):
    n = len(items)

    # Sort items by profit/weight ratio (descending)
    items.sort(key=lambda x: x[0]/x[1], reverse=True)

    Q = []  # priority queue
    u = Node(-1, 0, 0, 0)
    v = Node(0, 0, 0, 0)
    u.bound = bound(u, n, W, items)
    max_profit = 0

    heapq.heappush(Q, (-u.bound, u))  # push root node

    while Q:
        _, u = heapq.heappop(Q)

        if u.level == n - 1:
            continue

        # Next level node (include next item)
        v.level = u.level + 1
        v.weight = u.weight + items[v.level][1]
        v.profit = u.profit + items[v.level][0]

        # If within capacity and profit better, update max_profit
        if v.weight <= W and v.profit > max_profit:
            max_profit = v.profit

        # Compute bound for including next item
        v.bound = bound(v, n, W, items)

        # If promising, add to queue
        if v.bound > max_profit:
            heapq.heappush(Q, (-v.bound, Node(v.level, v.profit, v.weight, v.bound)))

        # Exclude next item
        v.weight = u.weight
        v.profit = u.profit
        v.bound = bound(v, n, W, items)
        if v.bound > max_profit:
            heapq.heappush(Q, (-v.bound, Node(v.level, v.profit, v.weight, v.bound)))

    return max_profit


# Example input
items = [(40, 2), (30, 5), (50, 10), (10, 5)]  # (profit, weight)
W = 16

print("Maximum Profit:", knapsackLC(items, W))
